#Anime recommendation Chatbot App with ChatGPT, LangChain and Streamlit

#Install Dependencies

In [ ]:
# Install necessary packages using pip
!pip install scikit-surprise
!pip install --upgrade scikit-learn
!pip install streamlit==1.32.2
!pip install pyngrok==7.1.5
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install fuzzywuzzy

#Load OpenAI API Credentials

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

#Set Environment Variable

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

#Clone Github Repo

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/SomersInias/AI-Anime-Recommendation.git

#Write App Code Header

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
import re
from difflib import get_close_matches
from fuzzywuzzy import fuzz, process

# --- Load Models and Data with Caching ---
@st.cache_resource
def load_models_and_data():
    try:
        # Load the pre-trained models and components
        svd = joblib.load('/content/drive/MyDrive/svd_model.joblib')
        tfidf = joblib.load('/content/drive/MyDrive/tfidf_vectorizer.joblib')
        cosine_sim = joblib.load('/content/drive/MyDrive/cosine_sim_matrix.joblib')
        indices = joblib.load('/content/drive/MyDrive/indices.joblib')

        # Load datasets
        #anime_df = pd.read_csv('/content/drive/MyDrive/anime.csv')
        #rating_df = pd.read_csv('/content/drive/MyDrive/rating.csv')
        anime_df = pd.read_csv('/content/AI-Anime-Recommendation/datasets/anime.csv')
        rating_df = pd.read_csv('/content/AI-Anime-Recommendation/datasets/rating.csv')
        return svd, tfidf, cosine_sim, indices, anime_df, rating_df
    except FileNotFoundError:
        st.error("Please make sure you have uploaded the model files and data files to the Colab environment.")
        st.stop()

svd, tfidf, cosine_sim, indices, anime_df, rating_df = load_models_and_data()

# Data cleaning
anime_df.drop_duplicates(inplace=True)
rating_df.drop_duplicates(inplace=True)
rating_df = rating_df[rating_df['rating'] != -1]
anime_df['genre'] = anime_df['genre'].fillna('')
anime_df['name_lower'] = anime_df['name'].str.lower()  # Add lowercase name for matching

# --- Smart Title Matching Functions ---
@st.cache_data
def create_title_mapping():
    """Create dictionaries for quick title lookup and variant mapping"""
    # Create a lookup dictionary for fuzzy matching
    title_dict = {}
    for idx, row in anime_df.iterrows():
        title = row['name'].lower()
        title_dict[title] = row['name']

        # Add simplified versions without special characters or year info
        simple_title = re.sub(r'[:\(\)\[\]\{\}].*$', '', title).strip()
        if simple_title != title and len(simple_title) > 3:
            title_dict[simple_title] = row['name']

    # Create a dictionary of similar titles
    similar_titles = {}
    title_list = anime_df['name'].tolist()

    # Group anime by base name (ignoring season/year info)
    base_titles = {}
    for title in title_list:
        # Extract base name (remove season numbers, years, etc.)
        base = re.sub(r'(?:\s+\(?(?:season|part|movie).*|\s+\(\d{4}\)|\s+\d+(?:st|nd|rd|th).*|\s+:.*)', '', title.lower(), flags=re.IGNORECASE)
        if base not in base_titles:
            base_titles[base] = []
        base_titles[base].append(title)

    # Create dictionary of related titles
    for base, variants in base_titles.items():
        if len(variants) > 1:
            for title in variants:
                similar_titles[title.lower()] = variants

    return title_dict, similar_titles

title_dict, similar_titles = create_title_mapping()

def find_closest_anime(user_title, confidence_threshold=70):
    """
    Find the closest matching anime title using multiple techniques
    Returns: (exact_match, matched_title, confidence_score, alternatives)
    """
    user_title = user_title.lower().strip()

    # 1. Direct match
    if user_title in title_dict:
        return True, title_dict[user_title], 100, []

    # 2. Fuzzy matching with scoring
    matches = process.extract(user_title, title_dict.keys(), limit=5, scorer=fuzz.token_sort_ratio)
    best_match, best_score = matches[0]

    alternatives = []
    if best_score >= confidence_threshold:
        # Prepare alternatives list
        for match, score in matches[1:4]:
            if score >= confidence_threshold - 10:
                alternatives.append(title_dict[match])

        return False, title_dict[best_match], best_score, alternatives

    # 3. Check for franchise similarity
    # This handles cases like "Naruto" matching "Naruto Shippuden" or "Naruto: The Movie"
    for anime_base, variants in similar_titles.items():
        if (fuzz.partial_ratio(user_title, anime_base) > 90 or
            fuzz.token_sort_ratio(user_title, anime_base) > 85):
            # Sort by similarity to user input
            sorted_variants = sorted(variants,
                                    key=lambda x: fuzz.token_sort_ratio(user_title, x.lower()),
                                    reverse=True)
            # Filter to main series vs movies
            main_series = [v for v in sorted_variants if not any(term in v.lower() for term in ['movie', 'film', 'ova'])]
            if main_series and 'movie' not in user_title and 'film' not in user_title:
                best_variants = main_series[:3]
            else:
                best_variants = sorted_variants[:3]

            return False, best_variants[0], 85, best_variants[1:3]

    # 4. No good matches found
    return False, None, 0, []

def resolve_anime_title_with_llm(user_title, chatgpt):
    """Use the LLM to resolve ambiguous anime titles"""
    prompt_template = """
    The user is looking for an anime with the title: "{user_title}"

    Based on your knowledge, determine the most likely canonical or common name for this anime.
    If this is a misspelling or informal name for a well-known anime, provide the correct title.
    If this could refer to multiple related anime (like a franchise), list up to 3 possibilities in order of likelihood.

    Format:
    Main title: [most likely correct title]
    Alternatives: [alternative 1], [alternative 2], [alternative 3]

    If you're highly confident this isn't a real anime title, respond with:
    Main title: None
    Alternatives: None
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | chatgpt

    response = chain.invoke({"user_title": user_title})
    content = response.content

    main_title = None
    alternatives = []

    # Extract main title
    main_match = re.search(r'Main title:\s*(.+)(?:\n|$)', content)
    if main_match and 'none' not in main_match.group(1).lower():
        main_title = main_match.group(1).strip()

    # Extract alternatives
    alt_match = re.search(r'Alternatives:\s*(.+)(?:\n|$)', content)
    if alt_match and 'none' not in alt_match.group(1).lower():
        alt_text = alt_match.group(1)
        alternatives = [alt.strip() for alt in re.split(r',\s*', alt_text)]

    return main_title, alternatives

def get_anime_by_title(title, chatgpt=None):
    """
    Smart function to find an anime by title, handling typos and variants
    Returns: (anime_id, exact_title, message)
    """
    # Try to find a match using our fuzzy matching
    exact_match, matched_title, confidence, alternatives = find_closest_anime(title)

    if exact_match:
        # Direct match found
        anime_entry = anime_df[anime_df['name'] == matched_title]
        return anime_entry['anime_id'].values[0], matched_title, None

    elif matched_title and confidence >= 70:
        # Good fuzzy match
        anime_entry = anime_df[anime_df['name'] == matched_title]

        message = f"Note: '{title}' was not found exactly. Using closest match: '{matched_title}'"
        if alternatives:
            alt_text = ", ".join([f"'{alt}'" for alt in alternatives])
            message += f"\nOther similar titles: {alt_text}"

        return anime_entry['anime_id'].values[0], matched_title, message

    elif chatgpt:
        # Try using LLM for more intelligent matching
        llm_title, llm_alternatives = resolve_anime_title_with_llm(title, chatgpt)

        if llm_title:
            # Try exact match with LLM suggestion
            anime_entry = anime_df[anime_df['name'] == llm_title]
            if not anime_entry.empty:
                message = f"Note: '{title}' was not found. Using LLM suggestion: '{llm_title}'"
                return anime_entry['anime_id'].values[0], llm_title, message

            # Try fuzzy match with LLM suggestion
            exact_match, matched_title, confidence, _ = find_closest_anime(llm_title)
            if matched_title and confidence >= 70:
                anime_entry = anime_df[anime_df['name'] == matched_title]
                message = f"Note: '{title}' was not found. Using closest match to LLM suggestion: '{matched_title}'"
                return anime_entry['anime_id'].values[0], matched_title, message

            # Try alternatives from LLM
            for alt_title in llm_alternatives:
                anime_entry = anime_df[anime_df['name'] == alt_title]
                if not anime_entry.empty:
                    message = f"Note: '{title}' was not found. Using LLM alternative suggestion: '{alt_title}'"
                    return anime_entry['anime_id'].values[0], alt_title, message

    # No good match found
    return None, None, f"Could not find a matching anime for '{title}'"

# --- Enhanced Recommendation Functions ---
def get_svd_recommendations(user_id, n=10):
    all_anime_ids = rating_df['anime_id'].unique()
    rated_anime = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    candidates = [aid for aid in all_anime_ids if aid not in rated_anime]
    predictions = [(aid, svd.predict(user_id, aid).est) for aid in candidates]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_n = predictions[:n]
    return pd.DataFrame([
        {'anime_id': aid, 'name': anime_df.loc[anime_df['anime_id'] == aid, 'name'].values[0], 'predicted_rating': pred}
        for aid, pred in top_n if len(anime_df.loc[anime_df['anime_id'] == aid, 'name'].values) > 0
    ])

def get_content_recommendations(title, n=10, chatgpt=None):
    # Use our smart title lookup
    anime_id, matched_title, message = get_anime_by_title(title, chatgpt)

    if anime_id is None:
        return message  # Return error message

    # Check if the anime is in our content-based model indices
    if matched_title not in indices:
        return f"'{matched_title}' found in database but not in content recommendation model."

    idx = indices[matched_title]
    sim_scores = sorted(list(enumerate(cosine_sim[idx])), key=lambda x: x[1], reverse=True)[1:n+1]
    anime_indices = [i[0] for i in sim_scores]
    result_df = anime_df.iloc[anime_indices][['anime_id', 'name', 'genre', 'rating']]

    # Add the message about title matching if needed
    if message:
        return message, result_df
    return result_df

def hybrid_recommendations(user_id, anime_title, alpha=0.5, n=10, chatgpt=None):
    # Get collaborative filtering recommendations
    svd_recs = get_svd_recommendations(user_id, n=500)

    # Find the anime using smart matching
    anime_id, matched_title, message = get_anime_by_title(anime_title, chatgpt)

    if anime_id is None:
        return message  # Return error message

    # Check if the anime is in our content-based model indices
    if matched_title not in indices:
        return f"'{matched_title}' found in database but not in content recommendation model."

    title_idx = indices[matched_title]
    scores = []

    for _, row in svd_recs.iterrows():
        candidate_idx = anime_df[anime_df['anime_id'] == row['anime_id']].index
        if len(candidate_idx) > 0:
            candidate_idx = candidate_idx[0]
            content_score = cosine_sim[title_idx][candidate_idx]
            collab_score = (row['predicted_rating'] - 1) / 9
            hybrid_score = alpha * collab_score + (1 - alpha) * content_score
            scores.append((row['anime_id'], row['name'], hybrid_score))

    scores.sort(key=lambda x: x[2], reverse=True)
    top_n = scores[:n]
    result_df = pd.DataFrame(top_n, columns=['anime_id', 'name', 'hybrid_score'])

    # Add the message about title matching if needed
    if message:
        return message, result_df
    return result_df

def create_test_user(anime_ratings, user_id=None, chatgpt=None):
    global rating_df
    if user_id is None:
        if rating_df.empty:
            user_id = 1
        else:
            user_id = rating_df['user_id'].max() + 1

    new_ratings = []
    not_found_titles = []
    matching_messages = []

    for title, rating in anime_ratings.items():
        # Use our smart title lookup
        anime_id, matched_title, message = get_anime_by_title(title, chatgpt)

        if anime_id is None:
            not_found_titles.append(title)
            continue

        new_ratings.append({
            'user_id': user_id,
            'anime_id': anime_id,
            'rating': rating
        })

        if message:
            matching_messages.append(message)

    # Check if we have any valid ratings
    if not new_ratings:
        return None, None, "No valid anime found for new user creation. Please try different anime titles."

    # Create and add the new ratings
    new_ratings_df = pd.DataFrame(new_ratings)
    rating_df = pd.concat([rating_df, new_ratings_df], ignore_index=True)

    # Prepare notification messages
    result_messages = []
    if matching_messages:
        result_messages.append("\n".join(matching_messages))

    if not_found_titles:
        not_found_msg = f"Could not find matches for: {', '.join(not_found_titles)}"
        result_messages.append(not_found_msg)

    final_message = "\n\n".join(result_messages) if result_messages else None

    return new_ratings_df, user_id, final_message

def parse_anime_ratings(input_text):
    """
    Parse anime ratings from various input formats.
    Supports single line inputs like "Naruto,9 One Piece,8" and multi-line inputs.
    """
    ratings_dict = {}
    error_message = None

    # First try to split by newlines if there are any
    if '\n' in input_text:
        lines = input_text.strip().split('\n')
    else:
        # If no newlines, try to split by spaces to identify rating pairs
        # This regex finds patterns like "Anime Title,8" or "Anime Title:8"
        rating_pattern = re.compile(r'([^,:\d]+)[,:](\d+\.?\d*)')
        matches = rating_pattern.findall(input_text)

        if matches:
            lines = []
            for title, rating in matches:
                lines.append(f"{title.strip()},{rating}")
        else:
            # If no clear pattern, treat as a single line
            lines = [input_text]

    # Process each line
    for line in lines:
        # Support both comma and colon separators
        if ',' in line:
            parts = line.split(',', 1)  # Split only at the first comma to handle anime titles with commas
        elif ':' in line:
            parts = line.split(':', 1)
        else:
            parts = []

        if len(parts) == 2:
            title = parts[0].strip()
            rating_str = parts[1].strip()

            # Extract just the number if there's extra text
            rating_match = re.search(r'(\d+\.?\d*)', rating_str)
            if rating_match:
                rating_str = rating_match.group(1)

            try:
                rating = float(rating_str)
                if 1 <= rating <= 10:  # Validate rating is in acceptable range
                    ratings_dict[title] = rating
                else:
                    error_message = f"Invalid rating value for '{title}': {rating_str}. Please use ratings between 1 and 10."
                    break
            except ValueError:
                error_message = f"Invalid rating format: '{rating_str}'. Please use numbers for ratings."
                break
        else:
            error_message = f"Invalid input format in '{line}'. Please use 'Title,Rating' format."
            break

    return ratings_dict, error_message

# --- Helper Functions for Beautiful Output ---
def format_recommendations_as_list(df, score_col=None):
    """Format recommendations dataframe as a nicely formatted list for display in Streamlit"""
    if df is None or df.empty:
        return "No recommendations found."

    # Create a string for the list of recommendations
    recommendations_list = ""

    for i, row in df.iterrows():
        anime_name = row['name']
        anime_id = row['anime_id']

        # Add score information if available
        score_info = ""
        if score_col and score_col in row:
            if score_col == 'predicted_rating':
                score = row[score_col]
                # Format score with one decimal place
                score_info = f" - Predicted rating: {score:.1f}/10"
            elif score_col == 'hybrid_score':
                # Convert hybrid score (0-1 scale) to a percentage
                score = row[score_col] * 100
                score_info = f" - Match score: {score:.1f}%"
            elif score_col == 'rating':
                # This is the average user rating
                score = row[score_col]
                score_info = f" - Average rating: {score:.1f}/10"

        # Add genre information if available
        genre_info = ""
        if 'genre' in row and row['genre']:
            genre_info = f" - Genres: {row['genre']}"

        # Add to recommendations list with nice formatting
        recommendations_list += f"**{i+1}. {anime_name}**{score_info}{genre_info}\n\n"

    return recommendations_list

# Create a function to display the ratings nicely
def format_ratings_as_list(df):
    """Format ratings dataframe as a nicely formatted list for display in Streamlit"""
    if df is None or df.empty:
        return "No ratings found."

    # Merge with anime_df to get anime names
    ratings_with_names = pd.merge(df, anime_df[['anime_id', 'name']], on='anime_id', how='left')

    # Create a string for the list of ratings
    ratings_list = ""

    for i, row in ratings_with_names.iterrows():
        anime_name = row['name']
        rating = row['rating']

        # Add to ratings list with nice formatting
        ratings_list += f"**{anime_name}**: {rating}/10\n\n"

    return ratings_list


# Initialize chat history
streamlit_msg_history = StreamlitChatMessageHistory(key="anime_chat_messages")
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Initialize session state for user creation flow
if 'awaiting_new_user_ratings' not in st.session_state:
    st.session_state.awaiting_new_user_ratings = False

# App title and sidebar
st.title("🌟 Anime Recommendation System")
st.sidebar.header("About")
st.sidebar.info(
    "This anime recommendation system uses collaborative filtering, "
    "content-based filtering, and hybrid approaches to suggest anime "
    "you might enjoy based on user ratings and anime attributes."
)

# Display initial welcome message
if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("Hello! How can I help you with anime recommendations today? You can ask for recommendations by user ID, by anime title, or a hybrid of both.")

# Display chat history
for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

# Get user input
if user_prompt := st.chat_input():
    st.chat_message("user").write(user_prompt)

    # --- Check if we're in the middle of creating a new user ---
    if st.session_state.awaiting_new_user_ratings:
        # Parse the ratings using our flexible parser
        ratings_dict, error_message = parse_anime_ratings(user_prompt)

        if error_message:
            ai_response_content = f"{error_message} Please try again."
        elif not ratings_dict:
            ai_response_content = "No valid anime titles and ratings detected. Please enter anime titles and ratings in the format 'Naruto,9 One Piece,8'."
        else:
            new_user_ratings_df, new_user_id, message = create_test_user(ratings_dict, chatgpt=chatgpt)
            if new_user_id is not None:
                # Successfully created user with at least some valid ratings
                recommendations_df = get_svd_recommendations(new_user_id, n=10)

                # Start with info about the new user
                ai_response_content = f"### New user created with User ID: {new_user_id} 🎉\n\n"

                # Add info about valid ratings
                ai_response_content += f"#### Successfully added {len(new_user_ratings_df)} ratings:\n\n"
                ai_response_content += format_ratings_as_list(new_user_ratings_df)

                # Add info about any matching/not found messages
                if message:
                    ai_response_content += f"#### Notes:\n{message}\n\n"

                # Add recommendations
                ai_response_content += f"### Here are anime recommendations for your new user profile:\n\n"
                ai_response_content += format_recommendations_as_list(recommendations_df, score_col='predicted_rating')
            else:
                # No valid ratings were added
                ai_response_content = message or "Failed to create new user."

        # Reset the state since we've processed the ratings
        st.session_state.awaiting_new_user_ratings = False

    else:
        # Normal flow - process intent
        intent_prompt_template = """
        You are an anime recommendation assistant. A user has provided a message, determine their intent.
        The possible intents are:

        get_user_recommendations: The user wants anime recommendations based on a user ID.
        get_anime_recommendations: The user wants anime recommendations based on an anime title.
        get_hybrid_recommendations: The user wants hybrid recommendations based on a user ID and an anime title.
        create_new_user: The user wants to create a new user profile and provide anime titles and ratings.
        unknown: The user's intent is not clear or doesn't fit into the above categories.

        Analyze the following user message and determine the intent. Respond with just the intent name (e.g., get_user_recommendations, unknown).

        User message: {message}
        """
        intent_prompt = ChatPromptTemplate.from_template(intent_prompt_template)
        intent_chain = intent_prompt | chatgpt

        # --- Parameter Extraction Chains ---
        user_id_extraction_prompt_template = """
        Extract the user ID from the following text. A user ID is typically a number.
        Look for a numerical value that is likely to be a user identifier.
        If you find a number that could be a user ID, extract ONLY the number.
        If no user ID is clearly mentioned or identifiable, respond with 'None'.

        Examples:
        - "recommendations for user 73517" -> "73517"
        - "user id 45 anime please" -> "45"
        - "hybrid for user: 123 and anime X" -> "123"
        - "no user id here" -> "None"

        Text: {text}
        """
        user_id_extraction_prompt = ChatPromptTemplate.from_template(user_id_extraction_prompt_template)
        user_id_extraction_chain = user_id_extraction_prompt | chatgpt

        anime_title_extraction_prompt_template = """
        Extract the anime title from the following text. The anime title is the name of an anime series.
        It could be multiple words. Try to identify the full title. If no anime title is found, respond with 'None'.

        Examples:
        - "recommend anime like Naruto" -> "Naruto"
        - "content based for One Piece" -> "One Piece"
        - "hybrid with Attack on Titan" -> "Attack on Titan"
        - "no anime mentioned" -> "None"

        Text: {text}
        """
        anime_title_extraction_prompt = ChatPromptTemplate.from_template(anime_title_extraction_prompt_template)
        anime_title_extraction_chain = anime_title_extraction_prompt | chatgpt

        # --- Process Intent ---
        intent_response = intent_chain.invoke({"message": user_prompt})
        intent = intent_response.content.strip()

        if intent == "get_user_recommendations":
            user_id_str = user_id_extraction_chain.invoke({"text": user_prompt}).content.strip()
            if user_id_str.lower() != 'none':
                try:
                    user_id = int(user_id_str)
                    if user_id <= 0:
                        ai_response_content = "Invalid user ID. User ID must be a positive number."
                    else:
                        recommendations_df = get_svd_recommendations(user_id, n=10)
                        if recommendations_df is not None and not recommendations_df.empty:
                            ai_response_content = f"### Anime Recommendations for User ID {user_id} 📺\n\n"
                            ai_response_content += format_recommendations_as_list(recommendations_df, score_col='predicted_rating')
                        else:
                            ai_response_content = f"No recommendations found for user ID {user_id} or user ID not found."
                except ValueError:
                    ai_response_content = "Invalid user ID format. Please provide a valid number for user ID."
            else:
                ai_response_content = "Please provide a user ID to get recommendations."

        elif intent == "get_anime_recommendations":
            anime_title = anime_title_extraction_chain.invoke({"text": user_prompt}).content.strip()
            if anime_title.lower() != 'none':
                result = get_content_recommendations(anime_title, n=10, chatgpt=chatgpt)

                if isinstance(result, tuple):
                    # We have a matching message and dataframe
                    matching_message, recommendations_df = result
                    ai_response_content = f"### Content-Based Recommendations for '{anime_title}' 🎬\n\n"
                    ai_response_content += f"**Note:** {matching_message}\n\n"
                    ai_response_content += format_recommendations_as_list(recommendations_df, score_col='rating')
                elif isinstance(result, pd.DataFrame):
                    # Just a dataframe
                    ai_response_content = f"### Content-Based Recommendations for '{anime_title}' 🎬\n\n"
                    ai_response_content += format_recommendations_as_list(result, score_col='rating')
                else:
                    # Error message
                    ai_response_content = result
            else:
                ai_response_content = "Please provide an anime title to get recommendations."

        elif intent == "get_hybrid_recommendations":
            user_id_str = user_id_extraction_chain.invoke({"text": user_prompt}).content.strip()
            anime_title = anime_title_extraction_chain.invoke({"text": user_prompt}).content.strip()

            user_id = None
            if user_id_str.lower() != 'none':
                try:
                    user_id = int(user_id_str)
                    if user_id <= 0:
                        ai_response_content = "Invalid user ID. User ID must be a positive number."
                        user_id = None
                except ValueError:
                    ai_response_content = "Invalid user ID format. Please provide a valid number for user ID."
                    user_id = None

            if anime_title.lower() == 'none':
                if not 'ai_response_content' in locals() or not ai_response_content:
                    ai_response_content = "Please provide both a user ID and an anime title for hybrid recommendations."
            elif user_id is not None and ('ai_response_content' not in locals() or not ai_response_content):
                result = hybrid_recommendations(user_id, anime_title, n=10, chatgpt=chatgpt)

                if isinstance(result, tuple):
                    # We have a matching message and dataframe
                    matching_message, recommendations_df = result
                    ai_response_content = f"### Hybrid Recommendations for User {user_id} based on '{anime_title}' 🌟\n\n"
                    ai_response_content += f"**Note:** {matching_message}\n\n"
                    ai_response_content += format_recommendations_as_list(recommendations_df, score_col='hybrid_score')
                elif isinstance(result, pd.DataFrame):
                    # Just a dataframe
                    ai_response_content = f"### Hybrid Recommendations for User {user_id} based on '{anime_title}' 🌟\n\n"
                    ai_response_content += format_recommendations_as_list(result, score_col='hybrid_score')
                else:
                    # Error message
                    ai_response_content = result

        elif intent == "create_new_user":
            ai_response_content = "### Let's create a new user profile! 👤\n\nPlease provide anime titles and your ratings (scale 1-10). You can enter them in a single line like:\n\n`Naruto,9 One Piece,8 Attack on Titan,10`\n\nOr on separate lines. I'll do my best to parse your input and find matching anime titles even if there are typos or variants."
            st.session_state.awaiting_new_user_ratings = True

        elif intent == "unknown":
            ai_response_content = "I'm sorry, I didn't understand what you want to do. Please clarify if you want recommendations by user ID, by anime title, hybrid recommendations, or to create a new user."

        else:
            ai_response_content = "Something went wrong processing your request. Please try again."

    # Add AI response to chat
    streamlit_msg_history.add_ai_message(ai_response_content)
    st.chat_message("ai").write(ai_response_content)

#Starting the Streamlit App

In [ ]:
!streamlit run app.py --server.port=8989 &>/./logs.txt &

#Setting Up ngrok Tunnel

In [ ]:
from getpass import getpass

ngrok_auth_token = getpass('Enter ngrok API Key: ')

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Authenticate ngrok with the token read from the file
!ngrok config add-authtoken {ngrok_auth_token}

# Open an HTTPS tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)